<a href="https://colab.research.google.com/github/nacho-gonz/NLP-Practica-archivos-2024/blob/main/TP1_2024/TP1_TUIA_2024_Procesamiento_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers

In [ ]:
import pandas as pd
from google.colab import files
from sentence_transformers import SentenceTransformer, util
import re
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
# Se guarda la URL de los libros
# Se realiza una solicitud HTTP para obtener el contenido de la página
url_libros = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'
response = requests.get(url_libros)
soup = BeautifulSoup(response.text, 'html.parser')

# Se navega por los diferentes "div" de la página para localizar la información de importancia
container_div = soup.find("div", class_="container")
libros_bloque = container_div.find("div", class_="page_content")

libros_links = libros_bloque.find('ol')

# Se crea una lista para almacenar los libros
links = []

# Si se encuentra información dentro de los links de los libros:
if libros_links:
    # Se itera dentro de cada link
    for li in libros_links.find_all("li"):
        # Encuentra la etiqueta donde está contenido el link de los diferentes libros
        a_tag = li.find("a")

        if a_tag and 'href' in a_tag.attrs:
            # Se obtiene el link y luego se le acopla al link base de los diferentes libros
            link = a_tag['href']
            url_cada_libro = 'https://www.gutenberg.org' + link

            # Realiza una solicitud para obtener el contenido de la página individual del libro
            response_libro = requests.get(url_cada_libro)
            soup_libro = BeautifulSoup(response_libro.text, 'html.parser')

            # Navegamos por la información del libro para guardar los datos más relevantes
            container_div_libro = soup_libro.find("div", class_="container")
            pagina_container_div_libro = container_div_libro.find("div", class_="page_content", id='content')
            pagina_body_div = pagina_container_div_libro.find("div", class_="page-body")
            tabs_wrapper_div = pagina_body_div.find("div", id="tabs-wrapper")
            bibrec_div = tabs_wrapper_div.find("div", id="bibrec")
            ebook_div = bibrec_div.find("div", {"typeof": "pgterms:ebook"})
            table = ebook_div.find("table", class_="bibrec")

            # Inicializa estructuras de datos para almacenar información sobre el libro
            datos = []
            informacion = {}
            subjects = []
            libro = {}

            # Extrae cada fila (<tr>) de la tabla y selecciona la etiqueta <th> (clave) y <td> (valor)
            for tr in table.find_all("tr"):
                th = tr.find("th")
                td = tr.find("td")

                if th and td:
                    key = th.text.strip()
                    value = td.text.strip()

                    # Almacena la información específica del libro en el diccionario
                    if key == 'Author':
                        informacion[key] = value
                    if key == 'Title':
                        informacion[key] = value
                    if key == 'Summary':
                        informacion[key] = value
                    if key == 'Subject':
                        subjects.append(value)  # Los temas se almacenan en una lista ya que para cada libro existe más de un tópico

            # Añade la lista de temas al diccionario de información del libro
            informacion['Subjects'] = subjects
            links.append(informacion)

            # Pausa de 1 segundo entre solicitudes para evitar el rechazo del servidor
            time.sleep(1)

# Se crea un DataFrame con la información guardada
# Se separa por "#" para evitar posibles problemas en un futuro
todos_libros = pd.DataFrame(links)
todos_libros.to_csv('todos_libros.csv', index=False, sep='#')

# Se descarga el DataFrame para que no sea necesario correr el código cada vez que se solicite la información
files.download("todos_libros.csv")

In [ ]:
modelo_semantic_multil = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
data_peliculas = pd.read_csv('https://github.com/nacho-gonz/NLP_TUIA_2024/raw/main/IMDB-Movie-Data.csv')
data_juegos_mesa = pd.read_csv('https://github.com/nacho-gonz/NLP_TUIA_2024/raw/main/bgg_database.csv', sep=',')
data_libros = pd.read_csv('https://github.com/nacho-gonz/NLP_TUIA_2024/raw/main/todos_libros.csv',sep='#')

In [ ]:
juegos_de_mesa = data_juegos_mesa.drop(labels=['rank','game_href','geek_rating','num_voters','best_num_players','maxplayers','maxplaytime','avgweight','minage', 'designers','avg_rating'],axis=1)
peliculas = data_peliculas.drop(labels=['Rank','Revenue (Millions)','Votes','Metascore','Rating'],axis=1)
peliculas = peliculas.rename(columns={'Genre':'categories', 'Description':'description'})
libros = data_libros.rename(columns={'Summary':'description', 'Subjects':'categories'})
libros.dropna(inplace=True,axis=0,subset=['description'])
libros.reset_index(drop=True, inplace=True)
libros['Author'] = libros['Author'].str.replace(r'\d+[-,]*', '', regex=True).str.strip()
libros['Author'] = libros['Author'].astype(str)

In [ ]:
juegos_de_mesa['embedding'] = 'Categorias: ' + juegos_de_mesa['categories']   + '. ' 'Descripción: ' + juegos_de_mesa['description']
juegos_de_mesa['embedding'] = juegos_de_mesa['embedding'].apply(lambda x: modelo_semantic_multil.encode(x, convert_to_tensor=True))

In [ ]:
libros['embedding'] = 'Categorias: ' + libros['categories']   + '. ' 'Descripción: ' + libros['description']
libros['embedding'] = libros['embedding'].apply(lambda x: modelo_semantic_multil.encode(x, convert_to_tensor=True))

In [ ]:
peliculas['embedding'] = 'Categorias: ' + peliculas['categories']   + '. ' 'Descripción: ' + peliculas['description']
peliculas['embedding'] = peliculas['embedding'].apply(lambda x: modelo_semantic_multil.encode(x, convert_to_tensor=True))

In [ ]:
libros['embedding_author'] = 'Author: ' + libros['Author']
libros['embedding_author'] = libros['embedding_author'].apply(lambda x: modelo_semantic_multil.encode(x, convert_to_tensor=True))

In [ ]:
peliculas['embedding_actor_director'] = 'Actors: ' + peliculas['Actors']   + '. ' 'Director: ' + peliculas['Director']
peliculas['embedding_actor_director'] = peliculas['embedding_actor_director'].apply(lambda x: modelo_semantic_multil.encode(x, convert_to_tensor=True))

In [ ]:
peliculas.to_csv('peliculas.csv',index=False)
juegos_de_mesa.to_csv('juegos_de_mesa.csv',index=False)
libros.to_csv('libros.csv',index=False)

In [ ]:
files.download('peliculas.csv')
files.download('juegos_de_mesa.csv')
files.download('libros.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
dataset = []

# Triste (0)
dataset.append((0, "Pensaba disfrutar mucho al aire libre, pero con lluvia se me quitaron las ganas."))
dataset.append((0, "Qué lástima que va a llover tantos días, no sé qué hacer."))
dataset.append((0, "Este clima lluvioso está arruinando mis planes de vacaciones."))
dataset.append((0, "Esperaba tomar sol, pero parece que solo veré nubes."))
dataset.append((0, "Estas vacaciones no están saliendo como esperaba con este clima."))
dataset.append((0, "Me siento un poco desanimado por las lluvias en mis vacaciones."))
dataset.append((0, "No puedo creer que justo me toca lluvia en mis días libres."))
dataset.append((0, "Con esta lluvia no me siento con ganas de hacer nada."))
dataset.append((0, "Quería descansar en la playa, pero este clima no ayuda."))
dataset.append((0, "Este clima me tiene bastante apagado, no sé qué hacer."))
dataset.append((0, "Todo lo que había planeado era para hacer al aire libre, ahora me siento perdido."))
dataset.append((0, "Tanta lluvia me está deprimiendo un poco en mis vacaciones."))
dataset.append((0, "Este clima me hace sentir un poco triste."))
dataset.append((0, "Esperaba sol, pero solo veo días grises y lluviosos."))
dataset.append((0, "Con tanta lluvia me siento un poco melancólico."))
dataset.append((0, "No me esperaba tanto mal clima, esto me deprime."))
dataset.append((0, "Este clima me da ganas de solo dormir todo el día."))
dataset.append((0, "Con esta lluvia se me fueron las ganas de hacer planes."))
dataset.append((0, "Estoy un poco decepcionado, no podré disfrutar la playa como quería."))
dataset.append((0, "La lluvia me está arruinando el ánimo en estas vacaciones."))
dataset.append((0, "Pensaba que serían unas vacaciones perfectas, pero me siento algo desilusionado."))
dataset.append((0, "Esperaba relajarme con el sol y el mar, pero esta lluvia me desanima."))
dataset.append((0, "Este clima no ayuda a que me sienta motivado."))
dataset.append((0, "La lluvia en vacaciones me hace sentir más cansado que feliz."))
dataset.append((0, "Tantos días grises en el lugar donde soñaba descansar me pone triste."))
dataset.append((0, "Quería días alegres en la playa, pero la lluvia me desmotiva."))
dataset.append((0, "Este clima nublado hace que mis vacaciones se sientan menos emocionantes."))
dataset.append((0, "Tenía ganas de hacer algo divertido, pero el clima me quita el ánimo."))
dataset.append((0, "No es la idea de vacaciones que tenía, la lluvia me hace sentir sin planes."))
dataset.append((0, "La lluvia me hace sentir un poco apagado, como si faltara algo."))
dataset.append((0, "No tengo muchas ganas de ver una película hoy, el clima me bajonea."))
dataset.append((0, "Quisiera disfrutar al aire libre, pero aquí estoy pensando si ver una película solo."))
dataset.append((0, "Leer un libro no me anima mucho, me siento algo desmotivado."))
dataset.append((0, "Me da igual qué hacer, no me siento animado ni para juegos ni películas."))
dataset.append((0, "Este día gris no me motiva para jugar ni leer, me siento un poco apagado."))
dataset.append((0, "Aunque intente ver una película, siento que no la disfrutaría hoy."))
dataset.append((0, "No tengo ánimos de nada, ni de leer ni de jugar con este clima."))
dataset.append((0, "No tengo ganas de hacer nada especial, solo me gustaría que deje de llover."))
dataset.append((0, "Siento que incluso ver una película no ayudaría a levantarme el ánimo."))
dataset.append((0, "La lluvia me quita las ganas de hacer cualquier cosa, solo quiero descansar."))
dataset.append((0, "No tengo motivación para juegos de mesa ni películas con este clima."))
dataset.append((0, "Pensé que una película me animaría, pero hoy nada parece entusiasmarme."))
dataset.append((0, "No creo que leer un libro ayude, me siento demasiado apagado."))
dataset.append((0, "Este clima gris me apaga las ganas de ver películas o jugar algo."))
dataset.append((0, "Con esta lluvia, siento que ni un buen libro puede animarme hoy."))
dataset.append((0, "No me siento de humor para una película ni para juegos de mesa."))
dataset.append((0, "Hoy no tengo ganas ni de leer ni de ver una película, la lluvia me desanima."))
dataset.append((0, "Los juegos de mesa se sienten aburridos en días como este."))
dataset.append((0, "No tengo energía para ver una película ni para leer un libro."))
dataset.append((0, "A veces ni siquiera un buen juego de mesa logra sacarme este ánimo."))
dataset.append((0, "Hoy no me siento para nada motivado para hacer algo."))
dataset.append((0, "Este clima arruinó mi humor para cualquier actividad divertida."))
dataset.append((0, "La lluvia me hace sentir que nada será lo suficientemente divertido."))
dataset.append((0, "No quiero hacer nada que requiera energía, como ver una película."))
dataset.append((0, "No creo que un libro o una película levanten este ánimo."))
dataset.append((0, "Me cuesta entusiasmarme, ni una película me llama la atención hoy."))
dataset.append((0, "Siento que no disfrutaría ninguna actividad, solo quiero que pare de llover."))
dataset.append((0, "El mal clima me quita las ganas de hacer algo como ver una película."))
dataset.append((0, "No me siento con ánimos para disfrutar ni siquiera un libro."))
dataset.append((0, "Hoy no tengo ganas de nada, ni películas ni libros."))
dataset.append((0, "A veces siento que estoy atrapado en una rutina que no me llena."))
dataset.append((0, "Me cuesta encontrarle sentido a mis días últimamente."))
dataset.append((0, "Siento que estoy perdiendo la motivación poco a poco."))
dataset.append((0, "Estoy agotado de seguir adelante sin ver resultados."))
dataset.append((0, "A veces siento que el esfuerzo no vale la pena."))
dataset.append((0, "Es difícil mantenerse positivo cuando todo parece igual."))
dataset.append((0, "Me invade una sensación de vacío que no logro explicar."))
dataset.append((0, "Siento que no tengo el control sobre mi propia vida."))
dataset.append((0, "Las cosas no están saliendo como las había planeado."))
dataset.append((0, "Me siento desconectado de las personas que me rodean."))
dataset.append((0, "Es frustrante sentir que no avanzo."))
dataset.append((0, "Siento que necesito un cambio, pero no sé por dónde empezar."))
dataset.append((0, "Me cuesta levantarme cada mañana para hacer lo mismo."))
dataset.append((0, "Estoy cansado de que mis esfuerzos no sean valorados."))
dataset.append((0, "Me siento atrapado en algo que ya no me hace feliz."))
dataset.append((0, "Es difícil sentirme animado cuando el trabajo me desgasta."))
dataset.append((0, "A veces no encuentro la energía para seguir."))
dataset.append((0, "Me siento solo, incluso rodeado de personas."))
dataset.append((0, "Cada día parece igual al anterior, sin nada que esperar."))
dataset.append((0, "La vida parece pesar más de lo que puedo cargar."))
dataset.append((0, "Siento que cada día me exige más de lo que puedo dar."))
dataset.append((0, "No logro encontrar la paz en mi rutina."))
dataset.append((0, "Me siento invisible, como si nada de lo que hago importara."))
dataset.append((0, "A veces solo quiero desaparecer por un momento."))
dataset.append((0, "Las cosas que antes me daban alegría ya no lo hacen."))
dataset.append((0, "Siento que me estoy perdiendo en el camino."))
dataset.append((0, "No encuentro un propósito claro que me motive."))
dataset.append((0, "Cada día me siento menos conectado con mis metas."))
dataset.append((0, "Es como si estuviera caminando en círculos, sin rumbo."))
dataset.append((0, "Me invade una tristeza que no logro identificar."))

# Neutral (1)
dataset.append((1, "Parece que habrá algunos días de lluvia, podré relajarme."))
dataset.append((1, "No puedo hacer todo lo que planeaba, pero siempre hay opciones para disfrutar."))
dataset.append((1, "Aunque llueva, intentaré aprovechar los días de descanso."))
dataset.append((1, "El clima no es el mejor, pero puedo hacer otras cosas para pasar el rato."))
dataset.append((1, "Aunque llueva, igual puedo descansar de la rutina."))
dataset.append((1, "No puedo ir a la playa, pero tengo algunas opciones en mente para estos días."))
dataset.append((1, "Con lluvia o sin ella, voy a aprovechar estos días libres."))
dataset.append((1, "A pesar de la lluvia, estoy contento de tener tiempo para mí."))
dataset.append((1, "Aunque no haga sol, puedo hacer algo diferente y descansar."))
dataset.append((1, "Con lluvia puedo disfrutar de otras formas, sin problemas."))
dataset.append((1, "No será lo que planeaba, pero encontraré una forma de disfrutar."))
dataset.append((1, "El clima está cambiante, pero trataré de hacer algo entretenido."))
dataset.append((1, "No siempre hace falta sol para pasar un buen rato."))
dataset.append((1, "No es lo ideal, pero tengo opciones para disfrutar de todas formas."))
dataset.append((1, "La lluvia cambiará algunos planes, pero buscaré alternativas."))
dataset.append((1, "Puedo aprovechar el mal clima para hacer otras cosas que disfruto."))
dataset.append((1, "El clima no es lo mejor, pero igual tengo opciones para pasarla bien."))
dataset.append((1, "Puedo hacer cosas diferentes que también pueden ser entretenidas."))
dataset.append((1, "Con días de lluvia, hay alternativas para descansar de otras formas."))
dataset.append((1, "No siempre se necesita buen clima para disfrutar de unos días libres."))
dataset.append((1, "Tendré que hacer algunos cambios, pero igual puedo disfrutar."))
dataset.append((1, "El clima cambia los planes, pero no las ganas de relajarme."))
dataset.append((1, "Puede que haga algo diferente a lo planeado, pero igual será bueno."))
dataset.append((1, "Con lluvia, puedo buscar algo para entretenerme sin problema."))
dataset.append((1, "No será lo que imaginaba, pero puedo relajarme de todos modos."))
dataset.append((1, "No todo será como esperaba, pero encontraré algo que me guste hacer."))
dataset.append((1, "Buscaré cómo entretenerme, aunque no pueda salir al aire libre."))
dataset.append((1, "El clima puede cambiar, pero no me preocupa."))
dataset.append((1, "No importa si llueve, siempre hay alternativas para disfrutar."))
dataset.append((1, "La lluvia no es ideal, pero puedo entretenerme con otras actividades."))
dataset.append((1, "Tal vez vea una película hoy para pasar el rato, aunque no tengo muchas expectativas."))
dataset.append((1, "Podría leer un poco, parece una buena forma de aprovechar la lluvia."))
dataset.append((1, "Pensaba en jugar algo de mesa, aunque no me importa mucho qué hacer."))
dataset.append((1, "Ver una película es una opción para este día, aunque no me entusiasma tanto."))
dataset.append((1, "El clima no es el mejor, pero podría jugar un juego de mesa o leer algo."))
dataset.append((1, "Quizá una película sea una buena idea, aunque tampoco me importaría leer un libro."))
dataset.append((1, "Hoy me da igual ver una película o leer, ambos pueden ser entretenidos."))
dataset.append((1, "Podría intentar un juego de mesa, aunque no tengo ninguna preferencia en mente."))
dataset.append((1, "No estoy muy emocionado, pero veré si encuentro un buen libro."))
dataset.append((1, "Puedo ver una película o jugar, no tengo preferencias."))
dataset.append((1, "Quizá una película no esté mal para hoy, aunque cualquier cosa me va."))
dataset.append((1, "Estoy abierto a ver una película o jugar, no tengo nada en especial en mente."))
dataset.append((1, "Leer podría ser interesante, pero no tengo mucha expectativa."))
dataset.append((1, "Podría jugar algo para pasar el rato, aunque no espero gran cosa."))
dataset.append((1, "Una película o un libro puede estar bien, aunque no me entusiasma mucho hoy."))
dataset.append((1, "Leer un libro puede ser entretenido, aunque tampoco me importa mucho."))
dataset.append((1, "No tengo mucho entusiasmo, pero un juego de mesa suena bien."))
dataset.append((1, "Podría ver algo tranquilo en la tele o leer, el clima es ideal para eso."))
dataset.append((1, "Hoy podría ser un buen día para un juego de mesa o una película."))
dataset.append((1, "Leer un libro me parece una opción cómoda para el día de hoy."))
dataset.append((1, "Podría ver una película, aunque no tengo muchas expectativas."))
dataset.append((1, "Tal vez lea un libro, aunque no tengo ninguna preferencia fuerte."))
dataset.append((1, "Voy a ver qué película hay, aunque tampoco tengo ganas de algo específico."))
dataset.append((1, "Un juego de mesa puede ser divertido, aunque me da un poco igual."))
dataset.append((1, "El clima es bueno para leer o ver una película, pero no tengo preferencia."))
dataset.append((1, "Podría leer un poco, aunque no me emociona mucho hoy."))
dataset.append((1, "Ver una película puede estar bien, aunque tampoco me motiva mucho."))
dataset.append((1, "Cualquiera de las opciones suena bien, hoy no tengo algo en mente."))
dataset.append((1, "Jugar, leer o ver una película, hoy cualquiera suena bien."))
dataset.append((1, "Hoy me da igual, puedo hacer cualquier actividad."))
dataset.append((1, "Es un día más, con la rutina de siempre."))
dataset.append((1, "Estoy tranquilo, aunque nada sobresaliente ha ocurrido."))
dataset.append((1, "Otro día que pasa sin demasiadas sorpresas."))
dataset.append((1, "La vida sigue su curso, sin grandes cambios."))
dataset.append((1, "Hoy es uno de esos días en los que todo está en su lugar."))
dataset.append((1, "Nada nuevo, solo el mismo ritmo de siempre."))
dataset.append((1, "A veces la rutina tiene su propia calma."))
dataset.append((1, "Siento que estoy en un momento estable de mi vida."))
dataset.append((1, "No hay altibajos hoy, solo paz y tranquilidad."))
dataset.append((1, "La rutina puede ser reconfortante en su propia manera."))
dataset.append((1, "No tengo nada de qué quejarme, ni tampoco algo que celebrar."))
dataset.append((1, "Hoy es un día sin sobresaltos ni preocupaciones."))
dataset.append((1, "Aprecio la calma de un día sin grandes eventos."))
dataset.append((1, "Es un día para seguir adelante, sin mucha prisa."))
dataset.append((1, "Nada especial, solo un día más para avanzar."))
dataset.append((1, "La vida sigue su curso, de manera tranquila."))
dataset.append((1, "Hoy me siento en paz con mi rutina."))
dataset.append((1, "La estabilidad también tiene su lado positivo."))
dataset.append((1, "La vida sigue en un tono neutral, sin nada nuevo."))
dataset.append((1, "Es un día típico, con sus ritmos y pausas habituales."))
dataset.append((1, "Hoy siento que todo está bajo control."))
dataset.append((1, "Un día que se siente como cualquier otro, en el buen sentido."))
dataset.append((1, "Es uno de esos días que pasan sin hacerse notar."))
dataset.append((1, "Hoy siento que todo está en equilibrio."))
dataset.append((1, "No hay mucho que resaltar, y está bien."))
dataset.append((1, "Siento que todo sigue su curso natural."))
dataset.append((1, "No es un día especial, pero no lo cambiaría."))
dataset.append((1, "Hoy todo parece estar en su sitio."))
dataset.append((1, "Siento paz con lo que hay, sin pedir más."))
dataset.append((1, "Hoy es un día común, y eso me da tranquilidad."))

# Alegre (2)
dataset.append((2, "Estoy emocionado por mis vacaciones, aunque llueva algunos días."))
dataset.append((2, "La lluvia no va a impedir que disfrute de estos días libres."))
dataset.append((2, "Estoy contento, me adaptaré al clima para pasarla bien igual."))
dataset.append((2, "¡No me importa la lluvia, estoy de vacaciones y eso me hace feliz!"))
dataset.append((2, "Voy a aprovechar cada minuto, llueva o no."))
dataset.append((2, "Estoy listo para disfrutar, incluso con mal clima."))
dataset.append((2, "¡Voy a divertirme igual aunque no pueda salir mucho!"))
dataset.append((2, "La lluvia también tiene su encanto en vacaciones."))
dataset.append((2, "Estoy de vacaciones y eso es lo que cuenta, ¡a disfrutar!"))
dataset.append((2, "Me emociona pensar en opciones como películas o juegos si llueve."))
dataset.append((2, "Tengo muchas ganas de descansar, no importa el clima."))
dataset.append((2, "El clima no arruinará mis planes, ¡haré lo mejor de cada día!"))
dataset.append((2, "¡Ya quiero leer ese libro que llevo tiempo guardando!"))
dataset.append((2, "Me siento feliz de estar libre, ¡llueva o no!"))
dataset.append((2, "Siempre hay maneras de pasarla bien, ¡incluso si llueve!"))
dataset.append((2, "Me encanta la idea de ver una película si no puedo salir."))
dataset.append((2, "Estar de vacaciones me hace sentir feliz, no importa el clima."))
dataset.append((2, "¡Estos días libres los voy a disfrutar al máximo, con o sin sol!"))
dataset.append((2, "No veo la hora de hacer algo divertido, ¡aunque llueva!"))
dataset.append((2, "Tengo opciones para entretenerme sin salir de casa, ¡genial!"))
dataset.append((2, "Llueva o truene, estos días serán para descansar y disfrutar."))
dataset.append((2, "Ver una película o jugar algo suena perfecto si llueve."))
dataset.append((2, "Estoy emocionado por cada opción, ¡no me arruinará el mal clima!"))
dataset.append((2, "¡Haré lo mejor de mis vacaciones, venga el clima que venga!"))
dataset.append((2, "El mal clima no me va a detener de disfrutar estos días libres."))
dataset.append((2, "¡Llueva o no, estos días son para mí, y me hace muy feliz!"))
dataset.append((2, "Siento que voy a disfrutar estos días de descanso como sea."))
dataset.append((2, "Ya planeé algunas cosas para hacer, ¡estoy emocionado!"))
dataset.append((2, "Estas vacaciones serán memorables, aunque llueva."))
dataset.append((2, "Con lluvia o sin ella, estoy listo para pasarla increíble."))
dataset.append((2, "¡Hoy es un buen día para ver una película y relajarme!"))
dataset.append((2, "Con lluvia afuera, leer un libro parece perfecto. ¡Qué emoción!"))
dataset.append((2, "Este clima es ideal para una buena tarde de juegos de mesa."))
dataset.append((2, "¡Voy a ver una película que quería ver hace tiempo!"))
dataset.append((2, "Siento que un juego de mesa con amigos sería genial para hoy."))
dataset.append((2, "¡Qué emoción! Este día lluvioso es ideal para leer ese libro nuevo."))
dataset.append((2, "Ver una película con palomitas suena perfecto para este clima."))
dataset.append((2, "Hoy voy a disfrutar una tarde de juegos, ¡me encanta esta opción!"))
dataset.append((2, "Estoy de humor para una película divertida, ¡será perfecto!"))
dataset.append((2, "Este clima me da ganas de leer algo inspirador, ¡qué buena idea!"))
dataset.append((2, "¡Qué buena excusa para una maratón de películas con lluvia afuera!"))
dataset.append((2, "Un juego de mesa en familia sería perfecto para esta tarde."))
dataset.append((2, "Hoy leeré un libro que me tiene atrapado, ¡no puedo esperar!"))
dataset.append((2, "¡Estoy listo para disfrutar de una buena película!"))
dataset.append((2, "¡Es un buen día para un maratón de lectura, estoy emocionado!"))
dataset.append((2, "Jugar un juego de mesa con amigos será perfecto para este día."))
dataset.append((2, "No puedo esperar a ver una película divertida, ¡qué buen plan!"))
dataset.append((2, "Voy a aprovechar para leer, ¡la lluvia lo hace perfecto!"))
dataset.append((2, "Hoy es ideal para un buen juego de mesa, ¡qué entretenido!"))
dataset.append((2, "Un día lluvioso como hoy es perfecto para una tarde de cine en casa."))
dataset.append((2, "Quiero leer algo emocionante mientras llueve, ¡qué lindo día!"))
dataset.append((2, "¡Qué buena oportunidad para una sesión de películas en casa!"))
dataset.append((2, "Me encanta el clima para jugar un buen juego de mesa con amigos."))
dataset.append((2, "Este día es perfecto para leer en paz, ¡me emociona la idea!"))
dataset.append((2, "¡Qué buen clima para ver una película y relajarme!"))
dataset.append((2, "Voy a disfrutar de un juego de mesa con todos, ¡qué divertido!"))
dataset.append((2, "Hoy es un gran día para sumergirme en una buena lectura."))
dataset.append((2, "Este clima es ideal para una maratón de películas, ¡no puedo esperar!"))
dataset.append((2, "Leer un libro mientras llueve afuera me hace feliz."))
dataset.append((2, "¡Hoy es el día perfecto para un juego de mesa familiar!"))
dataset.append((2, "Hoy siento que puedo lograr cualquier cosa que me proponga."))
dataset.append((2, "Me siento lleno de energía y entusiasmo por el día."))
dataset.append((2, "Es un día lleno de buenas vibras y optimismo."))
dataset.append((2, "Me encanta sentir que todo está saliendo bien."))
dataset.append((2, "Estoy disfrutando cada momento de este día."))
dataset.append((2, "Me siento agradecido por todo lo bueno que tengo."))
dataset.append((2, "Siento una energía positiva que me impulsa a hacer más."))
dataset.append((2, "Hoy estoy disfrutando al máximo cada pequeña cosa."))
dataset.append((2, "Tengo una sensación de alegría que me llena de paz."))
dataset.append((2, "Es un día de esos que se sienten realmente especiales."))
dataset.append((2, "Estoy lleno de gratitud por todas las cosas buenas."))
dataset.append((2, "Me siento motivado y con ganas de disfrutar cada instante."))
dataset.append((2, "Hoy todo parece brillar un poco más."))
dataset.append((2, "Siento que nada puede opacar este buen ánimo."))
dataset.append((2, "Tengo la energía para enfrentar cualquier cosa hoy."))
dataset.append((2, "Es un día perfecto para crear buenos recuerdos."))
dataset.append((2, "Estoy rodeado de buenas energías y alegría."))
dataset.append((2, "Cada momento de hoy parece estar lleno de magia."))
dataset.append((2, "Siento una gran paz y satisfacción en este momento."))
dataset.append((2, "Me siento contento, como si todo estuviera en su lugar."))
dataset.append((2, "Hoy todo parece estar fluyendo con facilidad."))
dataset.append((2, "Me siento afortunado de poder disfrutar este día."))
dataset.append((2, "Hoy siento que la vida me sonríe."))
dataset.append((2, "Es uno de esos días en los que todo parece posible."))
dataset.append((2, "Siento que hoy no hay límites para lo que puedo lograr."))
dataset.append((2, "Cada pequeño detalle me llena de alegría hoy."))
dataset.append((2, "Estoy disfrutando de la simplicidad de este día."))
dataset.append((2, "Hoy tengo la certeza de que la vida es hermosa."))
dataset.append((2, "Siento una conexión especial con todo lo que me rodea."))
dataset.append((2, "Estoy lleno de entusiasmo y ganas de aprovechar este día."))

In [ ]:
dataset = pd.DataFrame(dataset, columns=['label', 'sentence'])
dataset.to_csv('dataset_sent.csv', index=False)
files.download('dataset_sent.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>